In [ ]:
#hide
#all_slow

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
from fastai.text.all import *
from reformer_fastai.expscript import get_twin_sequence_dataloaders, get_lshlm_model, get_synthetic_learner
from reformer_fastai.data import MaskTargCallback

In [ ]:
#hide
#skip
torch.cuda.set_device(0)

In [ ]:
#hide
#skip
# models are saved in models/ **relative** to where script was run
# pm = Path('models/')
# pm.ls()

In [ ]:
#hide
#skip
#local mnodel filenames
fn1='synt_n_hashes-1_use-lsh-True_epochs-750_20210114_00:47:44'
fn2='synt_n_hashes-2_use-lsh-True_epochs-750_20210114_02:16:55'
fn4='synt_n_hashes-4_use-lsh-True_epochs-750_20210114_06:02:18'

## Get dataset

Settings from experiement. Note: possible error with seed setting. Test if 123 works.

In [ ]:
# Dataloaders
bs=64
sl=1024
train_sz=12800
valid_sz=1280
n_epochs=750
seed=123

In [ ]:
set_seed(seed, reproducible=True)
dls = get_twin_sequence_dataloaders(bs=bs, sl=sl, train_sz=train_sz, valid_sz=valid_sz, seed=seed)

## Load learners

In [ ]:
# Model
n_hashes=1
bucket_size=64  # suggested in trax
vocab_sz=128    # specific for the synthetic task
d_model=256
n_layers=1      # specified in paper
n_heads=4
d_ff=256

attn_dropout=0
ff_dropout=0
emb_dropout=0

max_seq_len=sl
causal=True
use_lsh=True

In [ ]:
def load_learner(n_hashes, fn):
    model = get_lshlm_model(vocab_sz=vocab_sz, d_model=d_model, n_layers=n_layers, n_heads=n_heads, 
              max_seq_len=max_seq_len, bucket_size=bucket_size, n_hashes=n_hashes, causal=causal, 
                use_lsh=use_lsh, seed=seed, attn_dropout=attn_dropout, ff_dropout=ff_dropout, emb_dropout=emb_dropout)
    learn = get_synthetic_learner(dls, model)
    learn = learn.load(fn)
    return learn

In [ ]:
learn_lsh1 = load_learner(n_hashes=1, fn=fn1)
learn_lsh2 = load_learner(n_hashes=2, fn=fn2)
learn_lsh4 = load_learner(n_hashes=4, fn=fn4)
learn_lsh1.model.n_hashes, learn_lsh2.model.n_hashes, learn_lsh4.model.n_hashes

(1, 2, 4)

## Validate with changing `n_hashes`:

In [ ]:
res = []
for learner in [learn_lsh4, learn_lsh2, learn_lsh1]:
    train_hashes = learner.model.n_hashes
    
    for eval_hashes in [8,4,2,1]:
        learner.model.n_hashes=eval_hashes
        _, m_acc = learner.validate(cbs=MaskTargCallback)
        res.append((train_hashes, eval_hashes, m_acc))

In [ ]:
cols = ['Training_hashes', 'Eval_hashes', 'Masked_Accuracy']
df = (pd.DataFrame(res, columns=cols)
      .pivot_table(index=cols[0],
                   columns=cols[1],
                   values=cols[2]))
df

Eval_hashes,1,2,4,8
Training_hashes,,,,
1,0.996561,0.999992,1.0,1.0
2,0.999945,1.000000,1.0,1.0
4,0.999521,1.000000,1.0,1.0


![table2](images/paper_table2.png)